In [1]:
import torch
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from glob import glob
import pandas as pd
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
import cv2
import numpy as np
from sklearn.svm import SVC
import torchvision.models as models

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
device

device(type='cuda')

In [4]:
imagePatches = glob('C:/Users/pedro/Downloads/archive/*/*/*')

In [5]:
len(imagePatches)

278082

In [6]:
imagePatches = [imagePatches[i] for i in range(len(imagePatches)) if 'IDC' not in imagePatches[i]]

In [7]:
len(imagePatches)

277524

In [8]:
y = []
for img in imagePatches:
    if img.endswith('class0.png'):
        y.append(0)
    elif img.endswith('class1.png'):
        y.append(1)

print(len(y))

277524


In [9]:
class MyDataset(Dataset):
    def __init__(self, df_data,transform=None):
        super().__init__()
        self.df = df_data.values

        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):
        img_path,label = self.df[index]

        image = cv2.imread(img_path)
        image = cv2.resize(image, (50,50))
        if self.transform is not None:
            image = self.transform(image)
        return image, label

In [10]:
images_df = pd.DataFrame()
images_df["images"] = imagePatches
images_df["labels"] = y
images_df.head()

,images,labels
0,C:/Users/pedro/Downloads/archive\10253\0\10253...,0
1,C:/Users/pedro/Downloads/archive\10253\0\10253...,0
2,C:/Users/pedro/Downloads/archive\10253\0\10253...,0
3,C:/Users/pedro/Downloads/archive\10253\0\10253...,0
4,C:/Users/pedro/Downloads/archive\10253\0\10253...,0


In [11]:
train, test = train_test_split(images_df, stratify=images_df.labels, test_size=0.2,random_state=42)
train, val = train_test_split(train, stratify=train.labels, test_size=0.2,random_state=42)
len(train), len(val),len(test)

(177615, 44404, 55505)

In [12]:
num_epochs = 20
num_classes = 2
batch_size = 128
learning_rate = 0.0001

In [13]:
trans_train = transforms.Compose([transforms.ToPILImage(),
                                  transforms.Pad(10, padding_mode='reflect'),
                                  transforms.RandomHorizontalFlip(),
                                  transforms.RandomVerticalFlip(),
                                  transforms.RandomRotation(20),
                                  transforms.ToTensor(),
                                  transforms.Normalize(mean=[0.5, 0.5, 0.5],std=[0.5, 0.5, 0.5])])

trans_valid = transforms.Compose([transforms.ToPILImage(),
                                  transforms.Pad(10, padding_mode='reflect'),
                                  transforms.ToTensor(),
                                  transforms.Normalize(mean=[0.5, 0.5, 0.5],std=[0.5, 0.5, 0.5])])

dataset_train = MyDataset(df_data=train, transform=trans_train)
dataset_valid = MyDataset(df_data=val,transform=trans_valid)
dataset_test = MyDataset(df_data=test,transform=trans_valid)

loader_train = DataLoader(dataset = dataset_train, batch_size=batch_size, shuffle=True, num_workers=0)
loader_valid = DataLoader(dataset = dataset_valid, batch_size=batch_size//2, shuffle=True, num_workers=0)
loader_test = DataLoader(dataset = dataset_test, batch_size=batch_size//2, shuffle=False, num_workers=0)

In [14]:
vggmodel = models.vgg16(weights='IMAGENET1K_V1')
vggmodel.classifier[6] = nn.Linear(4096, num_classes)

In [15]:
for n, p in vggmodel.named_parameters():
    print(p.requires_grad)

True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True


In [16]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(vggmodel.parameters(), lr=learning_rate, momentum=0.9)

In [17]:
vggmodel.to(device)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [18]:
vgg_best_accuracy = 0
vgg_best_weights = None

In [19]:
trl = []
trac = []
vall = []
valac = []

In [20]:
for epoch in range(num_epochs):
    train_loss = 0.0
    train_correct = 0
    train_total = 0
    vggmodel.train()
    for i, (inputs, targets) in enumerate(loader_train):
        inputs = inputs.to(device)
        targets = targets.to(device)
        optimizer.zero_grad()
        outputs = vggmodel(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        _, predicted = torch.max(outputs.data, 1)
        train_loss += loss.item() * inputs.size(0)
        train_correct += (predicted == targets).sum().item()
        train_total += targets.size(0)

    train_loss /= len(train)
    train_acc = train_correct / train_total

    trl.append(train_loss)
    trac.append(train_acc)


    val_loss = 0.0
    val_correct = 0
    val_total = 0
    vggmodel.eval()
    with torch.no_grad():
        for inputs, targets in loader_valid:
            inputs = inputs.to(device)
            targets = targets.to(device)
            outputs = vggmodel(inputs)
            loss = criterion(outputs, targets)
            _, predicted = torch.max(outputs.data, 1)
            val_loss += loss.item() * inputs.size(0)
            val_correct += (predicted == targets).sum().item()
            val_total += targets.size(0)

    val_loss /= len(val)
    val_acc = val_correct / val_total

    vall.append(val_loss)
    valac.append(val_acc)

    if val_acc > vgg_best_accuracy:
            vgg_best_accuracy = val_acc
            vgg_best_weights = vggmodel.state_dict()

    print(f"Epoch {epoch+1} - Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")

KeyboardInterrupt: 

In [ ]:
plt.plot(valac)
plt.plot(trac)
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['validation','train'])

In [ ]:
cuda_tensor = torch.tensor(vall)
vls = cuda_tensor.cpu()
cuda_tensor = torch.tensor(trl)
tls = cuda_tensor.cpu()

In [ ]:
plt.plot(vls)
plt.plot(tls)
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['validation','train'])

In [ ]:
vggmodel.load_state_dict(vgg_best_weights)
vggmodel.to(device)

In [ ]:
vggpredict = []
vgglabel = []

vggmodel.eval()
confusion_matrix = torch.zeros(2, 2)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in loader_test:
        images = images.to(device)
        labels = labels.to(device)
        outputs = vggmodel(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        vggpredict.extend(predicted)
        vgglabel.extend(labels)

        for t, p in zip(labels.view(-1), predicted.view(-1)):
                confusion_matrix[t.long(), p.long()] += 1

    print('Test Accuracy of the model on the test images: {} %'.format(100 * correct / total))

In [ ]:
label_vgg = [tensor.cpu().numpy() for tensor in vgglabel]
vgg_array = [tensor.cpu().numpy() for tensor in vggpredict]

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
print(confusion_matrix(label_vgg, vgg_array))
print(classification_report(label_vgg, vgg_array))

In [ ]:
dfv = pd.DataFrame()
dfv["vgg"] = vgg_array
dfv["label"] = label_vgg
dfv.head()
dfv.to_csv('vgwithaug.csv')

In [ ]:
torch.save({
    'model_state_dict': vggmodel.state_dict(),
    'optimizer_state_dict': optimizer.state_dict()
}, 'checkpointvgg50withaug.pth')